<a href="https://colab.research.google.com/github/duc-ke/study-book-keras_with_blocks/blob/master/3_imageDataGenerator_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://tykimos.github.io/warehouse/2017-3-8_CNN_Getting_Started_handwriting_shape.zip
!unzip 2017-3-8_CNN_Getting_Started_handwriting_shape.zip
!ls

--2020-03-02 07:13:25--  http://tykimos.github.io/warehouse/2017-3-8_CNN_Getting_Started_handwriting_shape.zip
Resolving tykimos.github.io (tykimos.github.io)... 185.199.111.153, 185.199.110.153, 185.199.109.153, ...
Connecting to tykimos.github.io (tykimos.github.io)|185.199.111.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tykimos.github.io/warehouse/2017-3-8_CNN_Getting_Started_handwriting_shape.zip [following]
--2020-03-02 07:13:25--  https://tykimos.github.io/warehouse/2017-3-8_CNN_Getting_Started_handwriting_shape.zip
Connecting to tykimos.github.io (tykimos.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 267133 (261K) [application/zip]
Saving to: ‘2017-3-8_CNN_Getting_Started_handwriting_shape.zip’

2017-3-8_CNN_Gettin 100%[===================>] 260.87K  --.-KB/s    in 0.005s  

2020-03-02 07:13:25 (46.8 MB/s) - ‘2017-3-8_CNN_Getting_Started_handwriting_shape.zip’ saved [

### 데이터 폴더 구조
![폴더구조](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2FcE6Im2%2FbtqCrjPGI0f%2FFk1ot9flkh8kYmnHQbRLOk%2Fimg.png)


In [2]:
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.models import Sequential

from keras.preprocessing.image import ImageDataGenerator    # augmentation 라이브러리!

import numpy as np

Using TensorFlow backend.


In [0]:
# 랜덤 시드 고정
np.random.seed(3)

### Image 데이터 준비 (ImageDataGenerator 이용, 폴더별 이미지 분류시 데이터셋 생성 기능 사용)
augmentation은 이번엔 사용하지 않음

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)    # 객체 생성
train_generator = train_datagen.flow_from_directory(
    'handwriting_shape/train/',
    target_size=(24, 24),    # 폴더에 있는 원본이미지 크기가 달라도 사이즈 자동 조절
    batch_size=3,
    class_mode='categorical'    # 'categorical' - 2D one hot(다중 클래스) / 'binary' 1D 이진 / 'sparse' 1D 정수 라벨 
)
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    'handwriting_shape/test/',
    target_size=(24, 24),
    batch_size=3,
    class_mode='categorical'
)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [0]:
def CNN():
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(24, 24, 3)))
  model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Flatten())
  
  model.add(Dense(128, activation='relu'))
  model.add(Dense(3, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

In [6]:
model = CNN()

In [7]:
# generator로 생성된 데이터셋은 fit 아닌 fit_generator

model.fit_generator(
    train_generator,
    steps_per_epoch=15,    # 한 epoch에서 사용한 스텝 수. 총 45개 train 데이터 + bath_size 3이므로 15스텝
    epochs=50,
    validation_data=test_generator,
    validation_steps=5    # 한 epoch종료 시 마다 검증할때 사용되는 검증 스텝수. 총 15개 test 데이터 + batch_size 3이므로 5
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/50





15/15 [==============================] - 14s 943ms/step - loss: 1.1600 - acc: 0.6222 - val_loss: 0.5925 - val_acc: 1.0000
Epoch 2/50
15/15 [==============================] - 0s 8ms/step - loss: 0.1864 - acc: 1.0000 - val_loss: 0.0621 - val_acc: 1.0000
Epoch 3/50
15/15 [==============================] - 0s 9ms/step - loss: 0.0107 - acc: 1.0000 - val_loss: 0.0244 - val_acc: 1.0000
Epoch 4/50
15/15 [==============================] - 0s 8ms/step - loss: 0.0088 - acc: 1.0000 - val_loss: 0.0891 - val_acc: 0.9333
Epoch 5/50
15/15 [==============================] - 0s 10ms/step - loss: 0.0110 - acc: 1.0000 - val_loss: 0.0249 - val_acc: 1.0000
Epoch 6/50
15/15 [==============================] - 0s 9ms/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0282 - val_acc: 1.0000
Epoch 7/50
15/15 [==============================] - 0s 9ms/step - loss: 9.4869e-04 - acc: 1.0000 - val_loss: 0.011

In [8]:
evaluation = model.evaluate(test_generator, steps=5)
print(evaluation)

5/5 [==============================] - 0s 4ms/step
[0.020033472860268374, 1.0]


In [15]:
# generator썼을땐 predict 아닌 predict_generator사용

output = model.predict_generator(test_generator, steps=5)
print(test_generator.class_indices)    # 각 클래스 이름을 알 수 있음
output

{'circle': 0, 'rectangle': 1, 'triangle': 2}


array([[0.000, 0.000, 1.000],
       [0.991, 0.000, 0.009],
       [0.000, 0.000, 1.000],
       [1.000, 0.000, 0.000],
       [0.000, 1.000, 0.000],
       [1.000, 0.000, 0.000],
       [0.000, 1.000, 0.000],
       [0.000, 1.000, 0.000],
       [0.000, 0.000, 1.000],
       [0.045, 0.748, 0.207],
       [0.000, 0.000, 1.000],
       [1.000, 0.000, 0.000],
       [1.000, 0.000, 0.000],
       [0.000, 0.000, 1.000],
       [0.000, 1.000, 0.000]], dtype=float32)

클래스별 확률값이 나타남

In [14]:
# 각 이미지 마다의 결과 출력 필요시. argmax이용
np.argmax(output, axis=1)

array([2, 0, 2, 0, 1, 0, 1, 1, 2, 1, 2, 0, 0, 2, 1])

In [0]:
# numpy의 수치형 포맷을 소숫점 3째 자리까지 나타내는 옵션
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [13]:
output

array([[0.000, 0.000, 1.000],
       [0.991, 0.000, 0.009],
       [0.000, 0.000, 1.000],
       [1.000, 0.000, 0.000],
       [0.000, 1.000, 0.000],
       [1.000, 0.000, 0.000],
       [0.000, 1.000, 0.000],
       [0.000, 1.000, 0.000],
       [0.000, 0.000, 1.000],
       [0.045, 0.748, 0.207],
       [0.000, 0.000, 1.000],
       [1.000, 0.000, 0.000],
       [1.000, 0.000, 0.000],
       [0.000, 0.000, 1.000],
       [0.000, 1.000, 0.000]], dtype=float32)